In [1]:
import findspark
from pyspark.sql import SparkSession

In [2]:
findspark.init()
spark = SparkSession.builder.appName("RetailByDayStructuredStreaming").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")

23/11/15 00:19:14 WARN Utils: Your hostname, bagjunhyeog-ui-noteubug.local resolves to a loopback address: 127.0.0.1; using 172.30.1.11 instead (on interface en0)
23/11/15 00:19:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 00:19:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("error")

In [4]:
staticDataFrame = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("./data/retail-data/by-day/*.csv")

In [5]:
from pyspark.sql import functions as func

In [6]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
preppedDataFrame = staticDataFrame \
    .na.fill(0) \
    .withColumn("day_of_week", func.date_format(func.col("InvoiceDate"), "EEEE")) \
    .coalesce(5)

In [8]:
trainDataFrame = preppedDataFrame \
    .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame \
    .where("InvoiceDate >= '2011-07-01'")

In [9]:
trainDataFrame.count()

245903

In [10]:
testDataFrame.count()

296006

- Spark MLLib은 일반적인 트랜스포메이션을 자동화하는 다양한 트랜스포메이션을 제공
- 그 중 하나가 `StringIndexer`

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer() \
    .setInputCol("day_of_week") \
    .setOutputCol("day_of_week_index")

- 위 예제는 요일 (`day_of_week`) 을 수치형으로 반환한다.
  > 예를 들어 토요일을 6으로 월요일을 1로
- 번호 지정 체계는 수치로 표현되어 토요일이 월요일보다 더 크다는 것을 의마하므로 잘못된 방식이다.
- 이를 보완하기 위해 `OneHotEncoder`를 사용 해서 각 값을 자체 컬럼으로 인코딩해야 한다.
- 이렇게 하면 특정 요일인지 아닌지 `Boolean` 타입으로 나타낼 수 있다.

In [13]:
from pyspark.ml.feature import OneHotEncoder

In [14]:
encoder = OneHotEncoder() \
    .setInputCol("day_of_week_index") \
    .setOutputCol("day_of_week_encoded")

- 스파크의 모든 머신러닝 알고리즘은 수치형 벡터 타입을 입력으로 사용한다.

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
vectorAssenbler = VectorAssembler() \
    .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"]) \
    .setOutputCol("features")

- 위 예제는 세 가지 핵심 특징인 가격(`UnitPrice`), 수량(`Quantity`), 특정 날짜의 요일(`day_ofweek_encoded`)을 가지고 있다.
- 다음은 나중에 입력값으로 들어올 데이터가 같은 프로세스를 거쳐 변환되도록 파이프라인을 설정하는 예제이다.


In [17]:
from pyspark.ml import Pipeline

In [18]:
transformationPipeline = Pipeline() \
    .setStages([indexer, encoder, vectorAssenbler])

- 학습 준비 과정은 두 단계로 이루어진다.
- 우선 transformer 를 데이터셋에 fit 시켜야 한다.
- 기본적으로 `StringIndelxer`는 인덱싱할 고윳값의 수를 알아야 한다.
- 고윳값을 수를 알 수 있다면 인코딩을 매우 쉽게 할 수 있지만, 만약 알 수 없다면 컬럼에 있는 모든 고윳값을 조사하고 인덱싱해야 한다.

In [19]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [20]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

- 데이터 캐싱을 이용해 중간 변환된 데이터셋의 복사본을 메모리로 저장해 전체 파이프라인을 재실행하는 것 보다 훨씬 빠르게 반복적으로 데이터셋에 접근할 수 있다.

In [21]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [22]:
from pyspark.ml.clustering import KMeans

In [23]:
kmeans = KMeans() \
    .setK(20) \
    .setSeed(1)

In [24]:
kmModel = kmeans.fit(transformedTraining)

In [27]:
transformedTest = fittedPipeline.transform(testDataFrame)

In [29]:
spark